In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
def black_scholes_merton_greeks(S, K, T, r, sigma, q, option_type='call'):
    """
    Calculate option price and Greeks using the Black-Scholes-Merton model.
    
    Returns:
    option_price, delta, gamma, vega, theta
    """
    
    # Calculate d1 and d2
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Calculate option price
    if option_type == 'call':
        option_price = (S * np.exp(-q * T) * norm.cdf(d1)) - (K * np.exp(-r * T) * norm.cdf(d2))
        delta = np.exp(-q * T) * norm.cdf(d1)
    elif option_type == 'put':
        option_price = (K * np.exp(-r * T) * norm.cdf(-d2)) - (S * np.exp(-q * T) * norm.cdf(-d1))
        delta = -np.exp(-q * T) * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")
    
    # Calculate Greeks
    gamma = (np.exp(-q * T) * norm.pdf(d1)) / (S * sigma * np.sqrt(T))
    vega = S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)
    
    if option_type == 'call':
        theta = -(S * sigma * np.exp(-q * T) * norm.pdf(d1)) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2) + q * S * np.exp(-q * T) * norm.cdf(d1)
    else:  # put
        theta = -(S * sigma * np.exp(-q * T) * norm.pdf(d1)) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2) - q * S * np.exp(-q * T) * norm.cdf(-d1)
    
    return option_price, delta, gamma, vega, theta

In [3]:
# Pricing for my current position
S =17032  # Underlying
K = 18600  # Strike price
DT = 34
T = DT/250  # Time to expiration (in years)
r = 0.0015  # Risk-free rate
sigma = 0.2157  # Volatility
q = 0.0424  # Dividend yield

call_results_P1 = black_scholes_merton_greeks(S, K, T, r, sigma, q, 'call')
#put_results_P1 = black_scholes_merton_greeks(S, K, T, r, sigma, q, 'put')

print("Call Option:")
print(f"Price: {call_results_P1[0]:.4f}")
print(f"Delta: {-call_results_P1[1]:.4f}")
print(f"Gamma: {-call_results_P1[2]:.6f}")
print(f"Vega: {-call_results_P1[3]:.4f}")
print(f"Theta: {-call_results_P1[4]:.4f}")

Call Option:
Price: 82.9508
Delta: -0.1270
Gamma: -0.000153
Vega: -1304.9140
Theta: 946.2386


In [4]:
#Constant for risk measurement
Delta_perc=0.01
Multiplier=10
Leveraged_Factor=2

In [15]:
#7200 HK
HSI_2X_Volume=800
HSI_2X_Price=2.972
HSI_2X_Value=HSI_2X_Volume*HSI_2X_Price
HSI_2X_Delta=HSI_2X_Value*Delta_perc*Leveraged_Factor
print("HSI_2X_Delta:", HSI_2X_Delta)

#3110 HK
HSI_HD_Vlume=200
HSI_HD_Price=21.94
HSI_HD_Value=HSI_HD_Vlume*HSI_HD_Price
HSI_HD_Delta=HSI_HD_Value*Delta_perc
print("HSI_HD_Delta:",HSI_HD_Delta )

#3037 HK
HSI_Volume=500
HSI_Price=17.69
HSI_Value=HSI_Volume*HSI_Price
HSI_Delta=HSI_Value*Delta_perc
print("HSI_Delta:", HSI_Delta)

#2837 HK
HSI_Tech_Volume=100
HSI_Tech_Price=4.23
HSI_Tech_Value=HSI_Tech_Volume*HSI_Tech_Price
HSI_Tech_Delta=HSI_Tech_Value*Delta_perc
print("HSI_Tech_Delta:",HSI_Tech_Delta)

HSI_2X_Delta: 47.552
HSI_HD_Delta: 43.88
HSI_Delta: 88.45
HSI_Tech_Delta: 4.23


In [13]:
#Total Delta from ETF
ETF_Total_Delta=HSI_2X_Delta+HSI_HD_Delta+HSI_Delta+HSI_Tech_Delta

#Options risk
Options_Port_Delta_Unit=-call_results_P1[1]
Options_Port_Delta_Dollar=-call_results_P1[1]*(S*Delta_perc)*Multiplier

#Combine ETF with Options
Net_Delta_Dollar=ETF_Total_Delta+Options_Port_Delta_Dollar
print("Net_Delta_Dollar:", Net_Delta_Dollar)

Unit_Dollar_Conversion=Options_Port_Delta_Dollar/Options_Port_Delta_Unit
Net_Delta_Unit=Net_Delta_Dollar/Unit_Dollar_Conversion
print("Net_Delta_Unit:", Net_Delta_Unit)

Net_Delta_Dollar: -32.15443420000926
Net_Delta_Unit: -0.01887883642555734


In [14]:
#################################Performance######################################
Capital_Input_ETF=20000
Capital_Input_Options=15000
Total_Input=Capital_Input_ETF+Capital_Input_Options

Capital_Output_ETF=19442.29
Capital_Output_Options=16267
Total_Output=Capital_Output_ETF+Capital_Output_Options

Net_PNL=Total_Output-Total_Input
print("Net_PNL:", Net_PNL)

Portfolio_Return=Net_PNL/Total_Input
print("Portfolio_Return:", Portfolio_Return)

Net_PNL: 709.2900000000009
Portfolio_Return: 0.020265428571428597


In [16]:
#########Hang Seng Index Performance###########
#Strategy Initition Date started on July 15th 2024
Benchmark_Initiation=18015
Benchmark_Now=17090
Benchmark_Return=(Benchmark_Now-Benchmark_Initiation)/Benchmark_Initiation
print("Benchmark_Return:", Benchmark_Return)

#Excess return over the benchmark (Hang Seng Index)
Excess_Return=Portfolio_Return-Benchmark_Return
print("Excess_Return:", Excess_Return)


Benchmark_Return: -0.05134610047182903
Excess_Return: 0.07161152904325763
